In [ ]:
from datetime import datetime, date, timedelta, time
import pandas as pd
import numpy as np
import logging
import json
import sqlite3
import yaml
import itertools
import os
import pytz
import hashlib
from shutil import copyfile

In [ ]:
df.drop(['year', 'month', 'day', 'AreaTypeCode', 'SubmissionTS'], axis=1, inplace=True)
df.rename(columns ={'ActualConsumption': 'consumption', 'ActualGenerationOutput': 'generation'},inplace=True)

In [ ]:
df.set_index(['MapCode','ProductionType_Name','AreaName'],append=True, inplace=True)

In [ ]:
df=df.unstack('MapCode')
df=df.unstack('ProductionType_Name')
df=df.unstack('AreaName')

In [ ]:
df=pd.concat([df], keys=['ENTSO-E'], names=['source', 'attribute', 'region', 'variable', 'subsource'],axis=1)

In [ ]:
df=pd.concat([df], keys= url, names=['web'], axis=1)

In [ ]:
df=df.reorder_levels(('region','variable','attribute','source','subsource','web'), axis=1)

In [ ]:
df

In [ ]:
colmap = {
        'ActualGenerationOutput': {
            'region': '{df.MapCode}',
            'variable': '{ProductionType_Name}',
            'attribute': 'generation',
            'source': 'ENTSO-E',
            'subsource': '{AreaName}',
            'web': url
        },
        'ActualConsumption': {
            'region': '{df.MapCode}',
            'variable': '{ProductionType_Name}',
            'attribute': 'consumption',
            'source': 'ENTSO-E',
            'subsource': '{AreaName}',
            'web': url
        }
    }

    # Drop any column not in colmap
#df = df[list(colmap.keys())]

    # Create the MultiIndex
#tuples = [tuple(colmap[col][level] for level in headers)
         # for col in df.columns]
df.columns = pd.MultiIndex.from_arrays([df.MapCode,df.ProductionType_Name,[],'ENTSO-E',df.AreaName, url], names=headers)

In [46]:
url='url'
filepath='C:/Users/Neon/Desktop/GitHub/download_entsoe/2017_2_AggregatedGenerationPerType.csv'
headers = ['region', 'variable', 'attribute', 'source', 'subsource', 'web']
variable_name='var'
df = pd.read_csv(
        filepath,
        sep='\t',
        encoding='utf-16',
        header=0,
        index_col='DateTime',
        parse_dates=True,
        nrows=20,
    )

In [47]:
colmap = {
        'ActualGenerationOutput': {
            'region': '{MapCode}',
            'variable': '{ProductionType_Name}',
            'attribute': 'generation',
            'source': 'ENTSO-E',
            'subsource': '{AreaName}',
            'web': url
        },
        'ActualConsumption': {
            'region': '{MapCode}',
            'variable': '{ProductionType_Name}',
            'attribute': 'consumption',
            'source': 'ENTSO-E',
            'subsource': '{AreaName}',
            'web': url
        }
    }

In [48]:
df.drop(['year', 'month', 'day', 'AreaTypeCode', 'SubmissionTS'], axis=1, inplace=True)

In [ ]:
tuples = [tuple(colmap[col][level].format(MapCode=MapCode, ProductionType_Name=ProductionType_Name, AreaName=AreaName)
for level in headers) for col in df.columns]

In [ ]:
df = df[[key for key in colmap.keys() if key in df.columns]]

In [ ]:
tuples

In [ ]:
df.columns = pd.MultiIndex.from_tuples(tuples, names=headers,)

In [44]:
tuples = [tuple(colmap[level].format(MapCode=col1,ProductionType_Name=col2,AreaName=col3)
                    for level in headers) for [col1, col2, col3] in df.columns]


KeyError: 'region'

In [ ]:
df.columns = pd.MultiIndex.from_tuples(tuples, names=headers)

In [45]:
colmap = {
        'ActualGenerationOutput': {
            'region': df.MapCode.values,
            'variable': df.ProductionType_Name.values,
            'attribute': 'generation',
            'source': 'ENTSO-E',
            'subsource': df.AreaName.values,
            'web': url
        },
        'ActualConsumption': {
            'region': df.MapCode.values,
            'variable': df.ProductionType_Name.values,
            'attribute': 'consumption',
            'source': 'ENTSO-E',
            'subsource': df.AreaName.values,
            'web': url
        }
    }

In [51]:
df

,AreaName,MapCode,ProductionType_Name,ActualConsumption,ActualGenerationOutput
DateTime,,,,,
2017-02-01 12:15:00,MAVIR CA,HU,Fossil Gas,0.00,1204.59
2017-02-01 12:15:00,MAVIR CA,HU,Other renewable,0.00,11.58
2017-02-01 18:15:00,MAVIR CA,HU,Nuclear,0.00,1415.40
2017-02-01 10:00:00,MAVIR CA,HU,Fossil Gas,0.00,1079.21
2017-02-01 11:45:00,MAVIR CA,HU,Fossil Gas,0.00,1076.86
2017-02-23 03:00:00,TransnetBW CA,DE_TransnetBW,Fossil Oil,5.68,0.00
2017-02-23 15:00:00,TransnetBW CA,DE_TransnetBW,Fossil Gas,0.00,0.00
2017-02-23 16:45:00,TransnetBW CA,DE_TransnetBW,Fossil Gas,0.00,0.00
2017-02-23 09:15:00,TransnetBW CA,DE_TransnetBW,Hydro Water Reservoir,0.00,62.60


In [ ]:
df = df[list(colmap.keys())]

In [53]:
tuples = [tuple(colmap[col].format(region=MapCode, ProductionType_Name=ProductionType_Name, AreaName=AreaName) for col in df.columns)]

KeyError: 'AreaName'

In [ ]:
df.columns = pd.MultiIndex.from_tuples(tuples, names=headers)

In [ ]:
df